# Network Science Final Project
Roberto & Gacera

## Data Collection
2019 election results is collected from jojie `/mnt/data/public/elections/nle2019/contests/`

In [1]:
import sqlite3
import pandas as pd
import glob
import re
import datetime
import pickle
import glob

### Get all 2019 election contest files

In [2]:
%%bash --out contests_file
find /mnt/data/public/elections/nle2019/contests/ -type f -name *.json

In [3]:
contests = [i for i in contests_file.split('\n') if i]

In [4]:
contests[0]

'/mnt/data/public/elections/nle2019/contests/4438.json'

In [5]:
values = []
for c in contests:
    with open(c, 'r') as f:
        data = json.load(f)
    for i in data['bos']:
        values.append((c, data['cc'], data['cn'], data['ccn'], i['boc'], i['bon'], i['pn']))

In [6]:
values[:2]

[('/mnt/data/public/elections/nle2019/contests/4438.json',
  4438,
  'MEMBER, SANGGUNIANG BAYAN ILOCOS NORTE - BANGUI   - LONE DIST',
  'COUNCILOR',
  20963,
  'ACOBA, ROGELIO (NP)',
  'NACIONALISTA PARTY'),
 ('/mnt/data/public/elections/nle2019/contests/4438.json',
  4438,
  'MEMBER, SANGGUNIANG BAYAN ILOCOS NORTE - BANGUI   - LONE DIST',
  'COUNCILOR',
  20964,
  'BALBAG, ROGERICK (NP)',
  'NACIONALISTA PARTY')]

In [7]:
conn = sqlite3.connect('elvotes2019.db')
conn.executescript('''
DROP TABLE IF EXISTS prep1_2019;
CREATE TABLE prep1_2019 (
    c_file      VARCHAR,
    cc          VARCHAR,
    cn          VARCHAR,
    ccn         VARCHAR,
    bo          VARCHAR,
    bon         VARCHAR,
    pn          VARCHAR
);
''')
conn.commit()

conn.executemany('''INSERT INTO prep1_2019 VALUES (?, ?, ?, ?, ?, ?, ?)''', values)
conn.commit()

In [8]:
df = pd.read_sql('SELECT * FROM prep1_2019', conn)

In [9]:
df.head()

,c_file,cc,cn,ccn,bo,bon,pn
0,/mnt/data/public/elections/nle2019/contests/44...,4438,"MEMBER, SANGGUNIANG BAYAN ILOCOS NORTE - BANGU...",COUNCILOR,20963,"ACOBA, ROGELIO (NP)",NACIONALISTA PARTY
1,/mnt/data/public/elections/nle2019/contests/44...,4438,"MEMBER, SANGGUNIANG BAYAN ILOCOS NORTE - BANGU...",COUNCILOR,20964,"BALBAG, ROGERICK (NP)",NACIONALISTA PARTY
2,/mnt/data/public/elections/nle2019/contests/44...,4438,"MEMBER, SANGGUNIANG BAYAN ILOCOS NORTE - BANGU...",COUNCILOR,20965,"CAMPAÑANO, ANTHONY (NP)",NACIONALISTA PARTY
3,/mnt/data/public/elections/nle2019/contests/44...,4438,"MEMBER, SANGGUNIANG BAYAN ILOCOS NORTE - BANGU...",COUNCILOR,20966,"DOLDOLEA, NORMA (NP)",NACIONALISTA PARTY
4,/mnt/data/public/elections/nle2019/contests/44...,4438,"MEMBER, SANGGUNIANG BAYAN ILOCOS NORTE - BANGU...",COUNCILOR,20968,"FAYLOGNA, SUSAN (NP)",NACIONALISTA PARTY


In [10]:
df.shape

(43451, 7)

### Filter positions needed for analysis

In [12]:
positions_needed = ['SENATOR', 'PARTY LIST', 'MEMBER, HOUSE OF REPRESENTATIVES',
                    'PROVINCIAL GOVERNOR', 'PROVINCIAL VICE-GOVERNOR',
                    'MAYOR', 'VICE-MAYOR']

In [13]:
df = df[df.ccn.isin(positions_needed)]

In [14]:
df.shape

(8843, 7)

### Get all 2019 election result files

In [15]:
coc_files = glob.glob('/mnt/data/public/elections/nle2019/results/*/*/*/coc.json')

In [16]:
cocs = [[i] + i[43:].split('/')[:-1] for i in coc_files]

In [17]:
cocs[0]

['/mnt/data/public/elections/nle2019/results/REGION I/ILOCOS NORTE/ADAMS/coc.json',
 'REGION I',
 'ILOCOS NORTE',
 'ADAMS']

In [18]:
len(cocs)

1655

In [19]:
conn.executescript('''
DROP TABLE IF EXISTS prep2_2019;
CREATE TABLE prep2_2019 (
    coc_file    VARCHAR,
    region      VARCHAR,
    province    VARCHAR,
    city        VARCHAR,
    cc          VARCHAR,
    bo          VARCHAR,
    v           VARCHAR,
    tot         VARCHAR,
    per         VARCHAR
);
''')
conn.commit()

i = 0
for c in cocs:
    with open(c[0], 'r') as file:
        data = json.load(file)
    for d in data['rs']:
        values = tuple(list(c) + [d['cc'], d['bo'], d['v'], d['tot'], d['per']])
        conn.execute('''INSERT INTO prep2_2019 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''', values)
    i+=1
    if i%100==0:
        conn.commit()
        print(datetime.datetime.now().time(), '- Finished', i, 'rows.')
conn.commit()

18:32:10.572379 - Finished 100 rows.
18:32:11.049598 - Finished 200 rows.
18:32:11.441989 - Finished 300 rows.
18:32:11.861411 - Finished 400 rows.
18:32:12.276664 - Finished 500 rows.
18:32:12.704843 - Finished 600 rows.
18:32:13.131572 - Finished 700 rows.
18:32:13.512401 - Finished 800 rows.
18:32:13.909030 - Finished 900 rows.
18:32:14.284618 - Finished 1000 rows.
18:32:14.645670 - Finished 1100 rows.
18:32:15.030448 - Finished 1200 rows.
18:32:15.398708 - Finished 1300 rows.
18:32:15.783509 - Finished 1400 rows.
18:32:16.122346 - Finished 1500 rows.
18:32:16.487400 - Finished 1600 rows.


In [20]:
df2 = pd.read_sql('SELECT * FROM prep2_2019', conn)

In [21]:
df2.shape

(392551, 9)

In [22]:
df2.tail()

,coc_file,region,province,city,cc,bo,v,tot,per
392546,/mnt/data/public/elections/nle2019/results/OAV...,OAV,EUROPE,ITALY,5567,43710,6,11629,0.05
392547,/mnt/data/public/elections/nle2019/results/OAV...,OAV,EUROPE,ITALY,5567,43711,6,11629,0.05
392548,/mnt/data/public/elections/nle2019/results/OAV...,OAV,EUROPE,ITALY,5567,43712,1,11629,0.00
392549,/mnt/data/public/elections/nle2019/results/OAV...,OAV,EUROPE,ITALY,5567,43713,15,11629,0.12
392550,/mnt/data/public/elections/nle2019/results/OAV...,OAV,EUROPE,ITALY,5567,43714,1149,11629,9.88


In [23]:
df.shape

(8843, 7)

In [24]:
df.tail()

,c_file,cc,cn,ccn,bo,bon,pn
43446,/mnt/data/public/elections/nle2019/contests/21...,2187,MAYOR NCR - TAGUIG CITY,MAYOR,6701,"CAYETANO, DIREK LINO (NP)",NACIONALISTA PARTY
43447,/mnt/data/public/elections/nle2019/contests/21...,2187,MAYOR NCR - TAGUIG CITY,MAYOR,6702,"CERAFICA, ARNEL (PDPLBN)",PARTIDO DEMOKRATIKO PILIPINO LAKAS NG BAYAN
43448,/mnt/data/public/elections/nle2019/contests/21...,2187,MAYOR NCR - TAGUIG CITY,MAYOR,6700,"ANDRADE, SONNY BOY (IND)",INDEPENDENT
43449,/mnt/data/public/elections/nle2019/contests/38...,3820,VICE-MAYOR NCR - TAGUIG CITY,VICE-MAYOR,10337,"CRUZ, RICARDO JR. (NP)",NACIONALISTA PARTY
43450,/mnt/data/public/elections/nle2019/contests/38...,3820,VICE-MAYOR NCR - TAGUIG CITY,VICE-MAYOR,10338,"DUEÑAS, JUN (PDPLBN)",PARTIDO DEMOKRATIKO PILIPINO LAKAS NG BAYAN


### Merge contest and results dataframes

In [25]:
df3 = pd.merge(df2, df)

In [26]:
df3.head(2)

,coc_file,region,province,city,cc,bo,v,tot,per,c_file,cn,ccn,bon,pn
0,/mnt/data/public/elections/nle2019/results/REG...,REGION I,ILOCOS NORTE,ADAMS,1,1,6,5366,0.11,/mnt/data/public/elections/nle2019/contests/1....,SENATOR PHILIPPINES,SENATOR,"ABEJO, VANGIE (IND)",LGBTQ PARTY
1,/mnt/data/public/elections/nle2019/results/REG...,REGION I,ILOCOS NORTE,BACARRA,1,1,89,87217,0.10,/mnt/data/public/elections/nle2019/contests/1....,SENATOR PHILIPPINES,SENATOR,"ABEJO, VANGIE (IND)",LGBTQ PARTY


In [27]:
df4 = df3[['c_file', 'coc_file', 'region', 'province', 'city', 'cn', 'ccn', 'bon', 'pn', 'v', 'tot', 'per']]

In [28]:
df4.to_sql('prep3_2019', conn, if_exists='replace', index=False)

In [29]:
df5 = df4.copy()

In [30]:
df5 = df5[['region', 'province', 'city', 'ccn', 'bon', 'pn', 'v', 'tot', 'per']]
df5.columns = ['region', 'province', 'city', 'position', 'candidate', 
               'candidate_party', 'votes', 'total_votes', 'percentage']

### Convert votes and total_votes to integer, percentage to float

In [31]:
df5.dtypes

region             object
province           object
city               object
position           object
candidate          object
candidate_party    object
votes              object
total_votes        object
percentage         object
dtype: object

In [32]:
df5.votes = df5.votes.astype(int)
df5.total_votes = df5.total_votes.astype(int)
df5.percentage = df5.percentage.astype(float)
df5.percentage = df5.percentage/100

In [38]:
df5.head(2)

,region,province,city,position,candidate,candidate_party,votes,total_votes,percentage
0,REGION I,ILOCOS NORTE,ADAMS,SENATOR,"ABEJO, VANGIE (IND)",LGBTQ PARTY,6,5366,0.0011
1,REGION I,ILOCOS NORTE,BACARRA,SENATOR,"ABEJO, VANGIE (IND)",LGBTQ PARTY,89,87217,0.0010


### Load final city-level 2019 election votes data to sqlite3 table and pickle file

In [39]:
df5.to_sql('city_votes_2019', conn, if_exists='replace', index=False)

In [40]:
with open('city_votes_2019.pkl', 'wb') as f:
    pickle.dump(df5, f)